In [1]:
import torch
import pickle
import os
import snntorch as snn
import torch.nn as nn
import numpy as np
from collections import OrderedDict

# PyTorch Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Function
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.optim import Adam
from torch.utils.data import random_split
from torch.utils.data import DataLoader, random_split
import torchvision

# Additional Imports
import snntorch as snn
import matplotlib.pyplot as plt
import numpy as np
import time
import os

# Dataset
import tonic
import tonic.transforms as transforms
from torch.utils.data import DataLoader
from tonic import DiskCachedDataset

# Network
import snntorch as snn
from snntorch import surrogate
from snntorch import functional as SF
from snntorch import spikeplot as splt
from snntorch import utils

import torchvision.transforms as torchvision_transforms
import tonic
import tonic.transforms as transforms
import json

In [4]:
# Define sensor size for NMNIST dataset
sensor_size = tonic.datasets.NMNIST.sensor_size

# Define transformations
# Note: The use of torch.from_numpy is removed as Tonic's transforms handle conversion.
transform = tonic.transforms.Compose([
    transforms.Denoise(filter_time=10000),
    transforms.ToFrame(sensor_size=sensor_size, time_window=10000),
    # torchvision.transforms.RandomRotation is not directly applicable to event data.
    # If rotation is needed, it should be done on the frames after conversion by ToFrame.
])

# Load NMNIST datasets without caching
trainset = tonic.datasets.NMNIST(save_to='/home/copparihollmann/neuroTUM/DATASETS/data/', transform=transform, train=True)
testset = tonic.datasets.NMNIST(save_to='/home/copparihollmann/neuroTUM/DATASETS/data', transform=transform, train=False)

# Split trainset into training and validation datasets
train_size = int(0.8 * len(trainset))
val_size = len(trainset) - train_size
train_dataset, val_dataset = random_split(trainset, [train_size, val_size])

# Create DataLoaders for training, validation, and testing
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=tonic.collation.PadTensors(batch_first=False))
val_loader = DataLoader(val_dataset, batch_size=batch_size, collate_fn=tonic.collation.PadTensors(batch_first=False))
test_loader = DataLoader(testset, batch_size=batch_size, collate_fn=tonic.collation.PadTensors(batch_first=False))

# Fetch a single batch from the train_loader to inspect the shape
data, targets = next(iter(train_loader))
print(f"Data shape: {data.shape}")  # Example output: torch.Size([batch_size, timesteps, channels, height, width])
print(f"Targets shape: {targets.shape}")  # Example output: torch.Size([batch_size])


Data shape: torch.Size([31, 128, 2, 34, 34])
Targets shape: torch.Size([128])


In [5]:
config = {
    # SNN
    "threshold1": 2.5,
    "threshold2": 8.0,
    "threshold3": 4.0,
    "threshold4": 2.0,
    "beta": 0.5,
    "num_steps": 10,
    
    # SNN Dense Shape
    "dense1_input": 2312,
    "num_classes": 10,
    

    # Network
    "batch_norm": True,
    "dropout": 0.3,

    # Hyper Params
    "lr": 0.007,

    # Early Stopping
    "min_delta": 1e-6,
    "patience_es": 20,

    # Training
    "epochs": 2
}

In [6]:
class SNN_tests(nn.Module):
  def __init__(self, config):
    super(SNN_tests, self).__init__()

    # Initialize configuration parameters
      # LIF
    self.thresh1 = config["threshold1"]
    self.thresh2 = config["threshold2"]
    self.thresh3 = config["threshold3"]
    self.beta = config["beta"]
    self.num_steps = config["num_steps"]

      # Dense Shape
    self.dense1_input = config["dense1_input"]
    self.num_classes = config["num_classes"]

      # Network Layers
    self.fc1 = nn.Linear(self.dense1_input, self.dense1_input//4)
    self.lif1 = snn.Leaky(beta=self.beta, threshold=self.thresh1)
    
    
    self.fc2 = nn.Linear(self.dense1_input//4, self.dense1_input//8)
    self.lif2 = snn.Leaky(beta=self.beta, threshold=self.thresh2)
    
    self.fc3 = nn.Linear(self.dense1_input//8, self.num_classes)
    self.lif3 = snn.Leaky(beta=self.beta, threshold=self.thresh3)
    
    self.flatten = nn.Flatten()
    
    
    # Forward Pass
  def forward(self, inpt):
    mem1 = self.lif1.init_leaky()
    mem2 = self.lif2.init_leaky()
    mem3 = self.lif3.init_leaky()

    all_outputs = {layer: [] for layer in ['inputs', 'fc1_outputs', 'lif1_spikes', 'fc2_outputs', 'lif2_spikes', 'fc3_outputs', 'lif3_spikes', 'mem1', 'mem2', 'mem3']}
    print(inpt.shape)
    for step in range(inpt.shape[0]):  # assuming inpt shape is (batch, time, ...)
        current_input = inpt[step, ...].to(device)
        
        current_input = self.flatten(current_input)

        current1 = self.fc1(current_input)
        spike1, mem1 = self.lif1(current1, mem1)
        current2 = self.fc2(spike1)
        spike2, mem2 = self.lif2(current2, mem2)
        current3 = self.fc3(spike2)
        spike3, mem3 = self.lif3(current3, mem3)

        # Collect outputs for each step
        all_outputs['inputs'].append(current_input.detach().cpu().numpy())
        all_outputs['fc1_outputs'].append(current1.detach().cpu().numpy())
        all_outputs['lif1_spikes'].append(spike1.detach().cpu().numpy())
        all_outputs['fc2_outputs'].append(current2.detach().cpu().numpy())
        all_outputs['lif2_spikes'].append(spike2.detach().cpu().numpy())
        all_outputs['fc3_outputs'].append(current3.detach().cpu().numpy())
        all_outputs['lif3_spikes'].append(spike3.detach().cpu().numpy())
        all_outputs['mem1'].append(mem1.detach().cpu().numpy())
        all_outputs['mem2'].append(mem2.detach().cpu().numpy())
        all_outputs['mem3'].append(mem3.detach().cpu().numpy())

    return all_outputs
  
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SNN_tests(config).to(device)

Save Intermediate results of the network for a fixed input 

In [5]:
import pickle

def load_and_save_model_outputs(model_path, test_loader, device):
    model = SNN_tests(config).to(device)
    model.eval()  # Set model to evaluation mode

    # Load the model weights
    model.load_state_dict(torch.load(model_path, map_location=device))

    # Grab a single sample from the loader
    single_sample, _ = next(iter(test_loader))  # Adjust if your dataloader is different
    single_sample = single_sample.to(device)

    # Run the forward pass
    layer_outputs = model(single_sample[:,0, ...].unsqueeze(1))  # Adjust slicing based on your data

    # Save the outputs
    save_layer_outputs(layer_outputs)

def save_layer_outputs(layer_outputs):
    base_dir = "./intermediate_outputs"
    if not os.path.exists(base_dir):
        os.makedirs(base_dir)
    
    for layer_name, timesteps_outputs in layer_outputs.items():
        layer_dir = os.path.join(base_dir, layer_name)
        if not os.path.exists(layer_dir):
            os.makedirs(layer_dir)
        
        for step, output in enumerate(timesteps_outputs):
            file_path = os.path.join(layer_dir, f"{layer_name}_timestep_{step}.bin")
            with open(file_path, 'wb') as f:
                pickle.dump(output, f)

# Assuming the model and device are defined, and model's forward() is correctly returning layer outputs
# Example Usage
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = "/home/copparihollmann/neuroTUM/SpikingC/SpikingCpp/notebooks/best_SNN_model.pth"
load_and_save_model_outputs(model_path, test_loader, device)


torch.Size([31, 1, 2, 34, 34])


In [9]:
import torch
import json
import os

def generate_metadata_from_model(model_path, test_loader, device, base_dir="./metadata"):
    model = SNN_tests(config).to(device)  # Make sure SNN_tests and config are defined/imported correctly
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    # Create directory for metadata if it does not exist
    if not os.path.exists(base_dir):
        os.makedirs(base_dir)

    # Initialize metadata dictionary
    metadata = {}

    # Process only one batch from the test loader
    single_sample, _ = next(iter(test_loader))
    single_sample = single_sample.to(device)

    # Forward pass to get the outputs
    with torch.no_grad():
        layer_outputs = model(single_sample[:, 0, ...].unsqueeze(1))  # Adjust based on your data's shape

    # Iterate through each layer's outputs and save metadata
    for layer_name, outputs in layer_outputs.items():
        # Example structure: layer_name might be 'fc1', outputs might be a tensor or a list of tensors
        if isinstance(outputs, list):
            shapes = [output.shape for output in outputs]
            dtypes = [str(output.dtype) for output in outputs]
        else:
            shapes = [outputs.shape]
            dtypes = [str(outputs.dtype)]
        
        # Store metadata
        metadata[layer_name] = {
            'shapes': shapes,
            'dtypes': dtypes
        }

    # Save metadata to a JSON file
    metadata_path = os.path.join(base_dir, "metadata.json")
    with open(metadata_path, 'w') as f:
        json.dump(metadata, f, indent=4)

# Assuming the model and device are defined
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = "/home/copparihollmann/neuroTUM/fix/SpikingCpp/notebooks/best_SNN_model.pth"
test_loader = test_loader
generate_metadata_from_model(model_path, test_loader, device)



torch.Size([31, 1, 2, 34, 34])


Save model weight and biases to binary files

In [7]:
def save_weights_and_biases_to_binary(model_path, save_directory="weights_and_bias_binary"):
    # Load the model's state dictionary
    state_dict = torch.load(model_path, map_location=torch.device('cpu'))

    # Create the directory if it doesn't exist
    if not os.path.exists(save_directory):
        os.makedirs(save_directory)

    metadata = {}

    # Save each weight and bias parameter to a separate binary file and record metadata
    for name, param in state_dict.items():
        if 'weight' in name or 'bias' in name:
            file_path = os.path.join(save_directory, f"{name.replace('.', '_')}.bin")
            metadata[name] = {
                'shape': param.shape,
                'dtype': str(param.dtype)
            }
            # Open the file in binary write mode and save the tensor data
            with open(file_path, 'wb') as f:
                numpy_array = param.numpy()
                f.write(numpy_array.tobytes())

    # Save metadata to a JSON file
    with open(os.path.join(save_directory, "metadata.json"), 'w') as f:
        json.dump(metadata, f, indent=4)

# Example usage
model_path = "/home/copparihollmann/neuroTUM/SpikingC/SpikingCpp/notebooks/best_SNN_model.pth"
save_weights_and_biases_to_binary(model_path)